In [ ]:
%conda install geopandas

In [ ]:
import pandas as pd

import emission.core.get_database as edb
import emission.core.wrapper.entry as ecwe
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.storage.decorations.trip_queries as esdt
import emission.storage.decorations.timeline as esdl
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt
import scaffolding
from uuid import UUID

# %conda install geopandas

import geopandas as gpd
from collections import defaultdict
import shapely as shp
import matplotlib.pyplot as plt

%matplotlib inline

import folium
import folium.plugins as fpl
import numpy as np
import branca.element as bre
import functools


In [ ]:
year = None
month = None
# program = "study"
study_type = "study"
mode_of_interest = "e-bike"
include_test_users = False #NO TEST USERS

dynamic_labels = {
    "MODE": [
      {"value":"walk", "baseMode":"WALKING", "met_equivalent":"WALKING", "kgCo2PerKm": 0},
      {"value":"e-auto_rickshaw", "baseMode":"MOPED", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.085416859},
      {"value":"auto_rickshaw", "baseMode":"MOPED", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.231943784},
      {"value":"motorcycle", "baseMode":"MOPED", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.113143309},
      {"value":"e-bike", "baseMode":"E_BIKE", "met": {"ALL": {"range": [0, -1], "mets": 4.9}}, "kgCo2PerKm": 0.00728},
      {"value":"bike", "baseMode":"BICYCLING", "met_equivalent":"BICYCLING", "kgCo2PerKm": 0},
      {"value":"drove_alone", "baseMode":"CAR", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.22031},
      {"value":"shared_ride", "baseMode":"CAR", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.11015},
      {"value":"e_car_drove_alone", "baseMode":"E_CAR", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.08216},
      {"value":"e_car_shared_ride", "baseMode":"E_CAR", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.04108},
      {"value":"taxi", "baseMode":"TAXI", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.30741},
      {"value":"bus", "baseMode":"BUS", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.20727},
      {"value":"train", "baseMode":"TRAIN", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.12256},
      {"value":"free_shuttle", "baseMode":"BUS", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.20727},
      {"value":"air", "baseMode":"AIR", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.09975},
      {"value":"not_a_trip", "baseMode":"UNKNOWN", "met_equivalent":"UNKNOWN", "kgCo2PerKm": 0},
      {"value":"other", "baseMode":"OTHER", "met_equivalent":"UNKNOWN", "kgCo2PerKm": 0}
    ],
    "PURPOSE": [
      {"value":"home"},
      {"value":"work"},
      {"value":"at_work"},
      {"value":"school"},
      {"value":"transit_transfer"},
      {"value":"shopping"},
      {"value":"meal"},
      {"value":"pick_drop_person"},
      {"value":"pick_drop_item"},
      {"value":"personal_med"},
      {"value":"access_recreation"},
      {"value":"exercise"},
      {"value":"entertainment"},
      {"value":"religious"},
      {"value":"other"}
    ],
    "translations": {
      "en": {
        "walk": "Walk",
        "e-auto_rickshaw":"E-tuk tuk",
        "auto_rickshaw":"Tuk Tuk",
        "motorcycle":"Motorcycle",
        "e-bike": "E-bike",
        "bike": "Bicycle",
        "drove_alone": "Car Drove Alone",
        "shared_ride": "Car Shared Ride",
        "e_car_drove_alone": "E-Car Drove Alone",
        "e_car_shared_ride": "E-Car Shared Ride",
        "taxi": "Taxi/Loca/inDrive",
        "bus": "Bus",
        "train": "Train",
        "free_shuttle": "Free Shuttle",
        "air": "Airplane",
        "not_a_trip": "Not a trip",
        "home": "Home",
        "work": "To Work",
        "at_work": "At Work",
        "school": "School",
        "transit_transfer": "Transit transfer",
        "shopping": "Shopping",
        "meal": "Meal",
        "pick_drop_person": "Pick-up/ Drop off Person",
        "pick_drop_item": "Pick-up/ Drop off Item",
        "personal_med": "Personal/ Medical",
        "access_recreation": "Access Recreation",
        "exercise": "Recreation/ Exercise",
        "entertainment": "Entertainment/ Social",
        "religious": "Religious",
        "other": "Other"
      },
      "lo": {
        "walk": "ດ້ວຍການຍ່າງ",
        "e-auto_rickshaw":"ລົດ 3 ລໍ້ໄຟຟ້າ ຫລື ຕຸກຕຸກໄຟຟ້າ",
        "auto_rickshaw":"ເດີນທາດ້ວຍ ລົດຕຸກຕຸກ ຫລື ລົດສາມລໍ້",
        "motorcycle":"ລົດຈັກ",
        "e-bike": "ວຍລົດຈັກໄຟຟ້າ",
        "bike": "ລົດຖີບ",
        "drove_alone": "ເດີນທາງ ດ້ວຍລົດໃຫ່ຍ ເຊີ່ງເປັນລົດທີ່ຂັບເອງ",
        "shared_ride": "ເດີນທາງດ້ວຍລົດໃຫ່ຍ ຮ່ວມກັບລົດຄົນອຶ່ນ",
        "e_car_drove_alone": "ດ້ວຍການຂັບລົດໄຟຟ້າໄປເອງ",
        "e_car_shared_ride": "ດ້ວຍການຈ້າງລົດໄຟຟ້າໄປ",
        "taxi": "ແທັກຊີ",
        "bus": "ລົດເມ",
        "train": "ລົດໄຟ",
        "free_shuttle": "ລົດຮັບສົ່ງຟຣີ",
        "air": "ຍົນ",
        "not_a_trip": "ບໍ່ແມ່ນການເດີນທາງ",
        "home": "ບ້ານ",
        "work": "ໄປເຮັດວຽກ",
        "at_work": "ຢູ່ບ່ອນເຮັດວຽກ",
        "school": "ໄປໂຮງຮຽນ",
        "transit_transfer": "ການຖ່າຍໂອນການເດີນທາງ",
        "shopping": "ຊອບປິ້ງ",
        "meal": "ອາຫານ",
        "pick_drop_person": "ໄປຮັບ ຫລື ສົນ ຄົນ",
        "pick_drop_item": "ໄປຮັບ ຫລື ສົ່ງສິນຄ້າ",
        "personal_med": "ໄປຫາໝໍ",
        "access_recreation": "ເຂົ້າເຖິງການພັກຜ່ອນ",
        "exercise": "ພັກຜ່ອນ/ອອກກຳລັງກາຍ",
        "entertainment": "ບັນເທີງ/ສັງຄົມ",
        "religious": "ຈຸດປະສົງທາງສາດສະໜາ",
        "other": "ອື່ນໆ"
      }
    }
  }

## Load the programs

In [ ]:
# Split UUIDs by program
program_uuid_map = {}
for ue in edb.get_uuid_db().find():
    program = ue['user_email'].split("_")[0]
    if program in program_uuid_map.keys():
        program_uuid_map[program].append(str(ue['uuid']))
    else:
        print(f"Found new program {program}, creating new list")
        program_uuid_map[program] = []
        program_uuid_map[program].append(str(ue['uuid']))

uuid_program_list = []
for ue in edb.get_uuid_db().find():
    program = ue['user_email'].split("_")[0]
    uuid_program_list.append({"program": program, "opcode": ue["user_email"], "user_id_str": str(ue['uuid'])})

uuid_program_df = pd.DataFrame.from_dict(uuid_program_list)
uuid_program_df.head()

In [ ]:
#workaround for custom labels :)
def mapping_labels(dynamic_labels, label_type):
    if "translations" in dynamic_labels and "en" in dynamic_labels["translations"]:
        translations = dynamic_labels["translations"]["en"]
        dic_mapping = dict()

        def translate_labels(labels):
            translation_mapping = {}
            for label in labels:
                value = label["value"]
                translation = translations.get(value)
                translation_mapping[value] = translation
            return defaultdict(lambda: 'Other', translation_mapping)
        dic_mapping = translate_labels(dynamic_labels[label_type])
        print(dic_mapping)
        return dic_mapping

dic_re = mapping_labels(dynamic_labels, 'MODE')

dic_pur = mapping_labels(dynamic_labels, 'PURPOSE')


## Load the trip table

In [ ]:
program = "usaid_laos_ev"
expanded_ct, file_suffix, quality_text, debug_df = await scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            dynamic_labels=dynamic_labels,
                                                                            include_test_users=include_test_users)
tq = scaffolding.get_time_query(year, month)
all_trips = await scaffolding.load_all_participant_trips(program, tq, False)

# (year, month, program, study_type, dynamic_labels, dic_re, dic_pur=None, include_test_users=False)

In [ ]:
print(len(expanded_ct))

early_testing = expanded_ct[expanded_ct['start_ts'] < 1700438400]
print(len(early_testing))
print(early_testing.user_id.nunique())


expanded_ct = expanded_ct[expanded_ct['start_ts'] > 1700438400]
print(len(expanded_ct))
expanded_ct["user_id_str"] = expanded_ct.user_id.apply(lambda u: str(u))
print(len(expanded_ct.user_id_str.unique()))
expanded_ct.user_id_str.unique()

earlt_testing_all = all_trips[all_trips['start_ts'] < 1700438400]

all_trips = all_trips[all_trips['start_ts'] > 1700438400]
print(len(all_trips))
all_trips["user_id_str"] = all_trips.user_id.apply(lambda u: str(u))
print(len(all_trips.user_id_str.unique()))
# all_trips.user_id_str.unique()

In [ ]:
early_testing.user_id.unique()

In [ ]:
earlt_testing_all.user_id.unique()

## Combine program and trip tables

In [ ]:
trip_program_df = expanded_ct.merge(uuid_program_df, on="user_id_str")
all_trip_df = all_trips.merge(uuid_program_df, on="user_id_str")

trip_program_df["distance_km"] = trip_program_df.distance / 1000
all_trip_df["distance_km"] = all_trip_df.distance / 1000

In [ ]:
trip_program_df.sample(n=50, random_state=123)[["program", "user_id_str", "opcode", "_id", "start_fmt_time", "end_fmt_time", "distance_km", "mode_confirm"]]
print(len(trip_program_df))
print(trip_program_df.user_id.nunique())

In [ ]:
all_trip_df.sample(n=50, random_state=123)[["program", "user_id_str", "opcode", "_id", "start_fmt_time", "end_fmt_time", "distance_km"]]
print(len(all_trip_df))
print(all_trip_df.user_id.nunique())

## Prepping dataframes for plotting

In [ ]:
def all_start_end(points):
    return points.start_loc.append(points.end_loc)

def geo_start_end(start_end):
    return gpd.GeoSeries(start_end.apply(lambda p: shp.geometry.Point(p['coordinates'])), crs="EPSG:4269")

In [ ]:
print(trip_program_df.user_id.nunique())
print(all_trip_df.user_id.nunique())

all_start_end_points = all_start_end(trip_program_df)
all_modeless_points = all_start_end(all_trip_df)

all_geo_start_end_points = geo_start_end(all_start_end_points)
all_geo_modeless = geo_start_end(all_modeless_points)

earlt_testing_all_points = all_start_end(earlt_testing_all)
earlt_testing_all_points_geo = geo_start_end(earlt_testing_all_points)

In [ ]:
trip_program_df.mode_confirm.unique()

In [ ]:
motorcycle_trips = trip_program_df[trip_program_df.mode_confirm == 'motorcycle']; 
print(len(motorcycle_trips))
print(motorcycle_trips.user_id.nunique())

motorcycle_start_end_points = all_start_end(motorcycle_trips); 
motorcycle_geo_start_end_points = geo_start_end(motorcycle_start_end_points)

car_like_trips = trip_program_df.query('mode_confirm == "own_car" | mode_confirm == "shared_ride" | mode_confirm == "drove_alone" | mode_confirm == "e_car_drove_alone" | mode_confirm == "e_car_shared_ride" | mode_confirm == "own_cars" | mode_confirm == "taxi"'); 
print(len(car_like_trips))
print(car_like_trips.user_id.nunique())

car_like_start_end_points = all_start_end(car_like_trips)
car_like_geo_start_end_points = geo_start_end(car_like_start_end_points)

In [ ]:
trip_program_df.Trip_purpose.unique()

In [ ]:
work_trips = trip_program_df[trip_program_df.Trip_purpose == 'To Work']; 
print(len(work_trips))
print(work_trips.user_id.nunique())

meal_shopping_trips = trip_program_df.query('Trip_purpose == "Shopping" | Trip_purpose == "Meal"'); 
print(len(meal_shopping_trips))
print(meal_shopping_trips.user_id.nunique())

school_trips = trip_program_df.query('Trip_purpose == "School"'); 
print(len(school_trips))
print(school_trips.user_id.nunique())

work_end_points = work_trips.end_loc; 
work_geo_end_points = geo_start_end(work_end_points)

meal_shopping_end_points = meal_shopping_trips.end_loc
meal_shopping_geo_end_points = geo_start_end(meal_shopping_end_points)

school_end_points = school_trips.end_loc
school_geo_end_points = geo_start_end(school_end_points)

### Heatmap using folium

Example from: https://stackoverflow.com/a/65756840/4040267

In [ ]:
def make_basic_heatmap(all_geo_start_end):

    map = folium.Map(
        location=(17.9757, 102.6331),
        zoom_start=11,
        control_scale = True
    )
    fpl.HeatMap(all_geo_start_end.apply(lambda p: [p.y, p.x])).add_to(map)
    return map

In [ ]:
#all labeled trips
make_basic_heatmap(all_geo_start_end_points)

In [ ]:
make_basic_heatmap(earlt_testing_all_points_geo)

In [ ]:
#all trips
make_basic_heatmap(all_geo_modeless)

In [ ]:
#TEST - scaled heatmap

import branca.colormap
from collections import defaultdict
import folium
import webbrowser
from folium.plugins import HeatMap 

def make_scaled_heatmap(data):
    map_osm = folium.Map(llocation=[17.9757, 102.6331],zoom_start=11, control_scale = True)

    steps=20
    colormap = branca.colormap.linear.Spectral_09.scale(0, 1).to_step(steps)
    colormap.colors.reverse()
    gradient_map=defaultdict(dict)
    for i in range(steps):
        gradient_map[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
    colormap.add_to(map_osm) #add color bar at the top of the map

    HeatMap(data.apply(lambda p: [p.y, p.x]),gradient = gradient_map).add_to(map_osm) # Add heat map to the previously created map

    return map_osm

map = make_scaled_heatmap(all_geo_modeless)
map

In [ ]:
#triple heatmap for all, car, motorcycle
all_map = folium.Map(
    location=(17.9757, 102.6331),
    zoom_start=11,
    control_scale = True
)
fpl.HeatMap(all_geo_start_end_points.apply(lambda p: [p.y, p.x])).add_to(all_map)

e_bike_map = folium.Map(
    location=(17.9757, 102.6331),
    zoom_start=11,
    control_scale = True
)
fpl.HeatMap(motorcycle_geo_start_end_points.apply(lambda p: [p.y, p.x])).add_to(e_bike_map)

car_like_map = folium.Map(
    location=(17.9757, 102.6331),
    zoom_start=11,
    control_scale = True
)
fpl.HeatMap(car_like_geo_start_end_points.apply(lambda p: [p.y, p.x])).add_to(car_like_map)


fig = bre.Figure()
fig.add_subplot(1,3,1).add_child(all_map)
fig.add_subplot(1,3,2).add_child(e_bike_map)
fig.add_subplot(1,3,3).add_child(car_like_map)

In [ ]:
make_basic_heatmap(work_geo_end_points)

In [ ]:
make_basic_heatmap(meal_shopping_geo_end_points)

In [ ]:
make_basic_heatmap(school_geo_end_points)

### Plot all e-bike trajectories (note that this will take several hours)

In [ ]:
def get_points_for_trip(user_id, trip_id):
    ts = esta.TimeSeries.get_time_series(user_id)
    return ts.get_data_df("analysis/recreated_location",
                         time_query=esda.get_time_query_for_trip_like("analysis/confirmed_trip", trip_id))

In [ ]:
def points_map(dataframe):
    
    #create a map
    this_map = folium.Map(prefer_canvas=True, control_scale = True)

    def plotDot(point):
        '''input: series that contains a numeric named latitude and a numeric named longitude
        this function creates a CircleMarker and adds it to your this_map'''
        folium.CircleMarker(location=[point.latitude, point.longitude],
                            radius=1,
                            weight=2).add_to(this_map)

    #use df.apply(,axis=1) to "iterate" through every row in your dataframe
    dataframe.apply(plotDot, axis = 1)

    #Set the zoom to the maximum possible
    this_map.fit_bounds(this_map.get_bounds())


    return this_map

In [ ]:
def points_map_objects(point_list):
    this_map = folium.Map(prefer_canvas=True, control_scale = True)

    def plotDot(point):
        '''input: series that contains a numeric named latitude and a numeric named longitude
        this function creates a CircleMarker and adds it to your this_map'''
        folium.CircleMarker(location=[point['coordinates'][1], point['coordinates'][0]],
                            radius=1,
                            weight=2).add_to(this_map)

    #use df.apply(,axis=1) to "iterate" through every row in your dataframe
    point_list.apply(plotDot)

    #Set the zoom to the maximum possible
    this_map.fit_bounds(this_map.get_bounds())

    return this_map

In [ ]:
def get_trajectory_points(trips):
    try:
        all_trip_points = trips.apply(lambda t: get_points_for_trip(t.user_id, t._id), axis=1)
    except: #once merged with surveys
        all_trip_points = trips.apply(lambda t: get_points_for_trip(t.user_id, t._id_x), axis=1)

    all_points = functools.reduce(lambda x, y: x.append(y), all_trip_points)

    all_points_gs = gpd.GeoSeries(all_points["loc"].apply(lambda p: shp.geometry.Point(p['coordinates'])), crs="EPSG:4269")

    return all_trip_points, all_points, all_points_gs

In [ ]:
points_map_objects(earlt_testing_all_points)

In [ ]:
#work trips
print(len(work_end_points))
points_map_objects(work_end_points)

In [ ]:
#shopping trips
print(len(meal_shopping_end_points))
points_map_objects(meal_shopping_end_points)

In [ ]:
#school trips
print(len(school_end_points))
points_map_objects(school_end_points)

In [ ]:
#using motorcycle for Laos application
e_bike_all_trip_points,  e_bike_all_points, e_bike_all_points_gs= get_trajectory_points(motorcycle_trips)

print(len(e_bike_all_points))
e_bike_all_points.head()

In [ ]:
points_map(e_bike_all_points)

In [ ]:
car_all_trip_points, car_all_points, car_all_points_gs = get_trajectory_points(car_like_trips)

print(len(car_all_points))
car_all_points.head()

In [ ]:
points_map(car_all_points)

# Gender Space!

In [ ]:
surveys = pd.read_csv('DemographicData.csv')
print(len(surveys), 'surveys')

#drop any null ids
socio_data = surveys[~surveys['user_id'].isnull()]
print(len(socio_data), 'surveys after dropping null ids')

#drop duplicates
socio_data = socio_data.sort_values(by=['user_id', 'data.ts'])
socio_data.drop_duplicates(subset=['user_id'], keep='last', inplace=True)
print(len(socio_data),'surveys', socio_data['user_id'].nunique(), 'users after dropping duplicates')

#prepare survey ids for merging
socio_data['user_id_socio'] = socio_data['user_id'].astype(str)
socio_data['user_id_socio'] = socio_data['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
socio_data['user_id_socio'] = [i.replace('-','') for i in socio_data.user_id_socio]
socio_data['user_id_socio'] = socio_data['user_id_socio']
socio_data = socio_data.drop(labels='user_id', axis=1)

In [ ]:
socio_data = socio_data[socio_data['data.local_dt.timezone'] != 'America/Denver'] #throw out the test codes
print(len(socio_data))

In [ ]:
tq = scaffolding.get_time_query(year, month)
all_trips = await scaffolding.load_all_participant_trips('usaid-laos-ev', tq, False)

all_trips = all_trips[all_trips['start_ts'] > 1700438400]
all_trips["user_id_str"] = all_trips.user_id.apply(lambda u: str(u))
len(all_trips.user_id_str.unique())

all_trip_df = all_trips.merge(uuid_program_df, on="user_id_str")
all_trip_df["distance_km"] = all_trip_df.distance / 1000
all_trip_df.head()

In [ ]:
#prepare trip ids for merging
trips = all_trip_df.copy()
print(len(trips), 'trips')
print(trips.user_id.nunique(), 'people')
trips['user_id_socio'] = trips.user_id.astype(str)
trips['user_id_socio'] = trips['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
trips.user_id_socio = [i.replace('-','') for i in trips.user_id_socio] # remove all dashes from strings

#merge the data
data = trips.merge(socio_data, on='user_id_socio')
print(len(data), 'trips after merging')
print(data.user_id_socio.nunique(), 'people after merging')

In [ ]:
data['GENDER'] = data['What_is_your_gender']

woman = data[data.GENDER == 'woman']
print(len(woman), "woman trips")
print(woman.user_id.nunique(), "women")
man = data[data.GENDER == 'man']
print(len(man), "man trips")
print(man.user_id.nunique(), "men")

In [ ]:
all_woman = all_start_end(woman)
all_geo_woman = geo_start_end(all_woman)
print(len(all_geo_woman))

all_man = all_start_end(man)
all_geo_man = geo_start_end(all_man)
print(len(all_geo_man))

In [ ]:
make_basic_heatmap(all_geo_woman)

In [ ]:
make_basic_heatmap(all_geo_man)

In [ ]:
points_map_objects(all_woman)

In [ ]:
points_map_objects(all_man)

## Gender + Mode

In [ ]:
expanded_ct, file_suffix, quality_text, debug_df = scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            dynamic_labels=dynamic_labels,
                                                                            dic_re=dic_re,
                                                                            dic_pur=dic_pur,
                                                                            include_test_users=include_test_users)

In [ ]:
#prepare trip ids for merging
trips = expanded_ct.copy()
print(len(trips), 'trips')
print(trips.user_id.nunique(), 'people')
trips['user_id_socio'] = trips.user_id.astype(str)
trips['user_id_socio'] = trips['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
trips.user_id_socio = [i.replace('-','') for i in trips.user_id_socio] # remove all dashes from strings

#merge the data
data = trips.merge(socio_data, on='user_id_socio')
print(len(data), 'trips after merging')
print(data.user_id_socio.nunique(), 'people after merging')

In [ ]:
data.columns

In [ ]:
motorcycle = data[data["mode_confirm"] == "motorcycle"]
print(len(motorcycle), "total motorcycle trips")

woman_motorcycle = motorcycle[motorcycle["What_is_your_gender"] == "woman"]
print(len(woman_motorcycle), "by women")
print(woman_motorcycle.user_id.nunique())

man_motorcycle = motorcycle[motorcycle["What_is_your_gender"] == "man"]
print(len(man_motorcycle), "by men")
print(man_motorcycle.user_id.nunique())

In [ ]:
all_woman_cycle = all_start_end(woman_motorcycle)
all_geo_woman_cycle = geo_start_end(all_woman_cycle)
print(len(all_geo_woman_cycle))

all_man_cycle = all_start_end(man_motorcycle)
all_geo_man_cycle = geo_start_end(all_man_cycle)
print(len(all_geo_man_cycle))

In [ ]:
woman_motorcycle.columns

In [ ]:
all_woman_cycle_trip_points, all_woman_cycle_points,  all_woman_cycle_points_gs = get_trajectory_points(woman_motorcycle)

print(len(all_woman_cycle_points))
all_woman_cycle_points.head()

In [ ]:
points_map(all_woman_cycle_points)

In [ ]:
all_man_cycle_trip_points, all_man_cycle_points, all_man_cycle_points_gs = get_trajectory_points(man_motorcycle)

print(len(all_man_cycle_points))
all_man_cycle_points.head()

In [ ]:
points_map(all_man_cycle_points)

# Gender + Purpose

In [ ]:
data.Trip_purpose.unique()

In [ ]:
work_trips = data[data['Trip_purpose'] == "To Work"]
print(len(work_trips), " total work trips")

woman_work = work_trips[work_trips["What_is_your_gender"] == "woman"]
print(len(woman_work), " by women")
print(woman_work.user_id.nunique())

man_work = work_trips[work_trips["What_is_your_gender"] == "man"]
print(len(man_work), "by men")
print(man_work.user_id.nunique())

In [ ]:
women_work_end_points = woman_work.end_loc; 
women_work_geo_end_points = geo_start_end(women_work_end_points)

men_work_end_points = man_work.end_loc; 
men_work_geo_end_points = geo_start_end(men_work_end_points)

In [ ]:
print(len(women_work_geo_end_points))
make_basic_heatmap(women_work_geo_end_points)

In [ ]:
print(len(men_work_geo_end_points))
make_basic_heatmap(men_work_geo_end_points)

## Experimenting with whether we can associate other fields with the start_end_loc points

In [ ]:
start_loc_df = trip_program_df[["program", "mode_confirm", "start_loc"]].copy()
start_loc_df["type"] = ["start"] * len(start_loc_df)
start_loc_df.rename(columns = {"start_loc": "location"}, inplace=True)
start_loc_df.head()

In [ ]:
end_loc_df = trip_program_df[["program", "mode_confirm", "end_loc"]].copy()
end_loc_df["type"] = ["end"] * len(end_loc_df)
end_loc_df.rename(columns = {"end_loc": "location"}, inplace=True)
end_loc_df.head()

In [ ]:
loc_program_df = gpd.GeoDataFrame(start_loc_df.append(end_loc_df))
loc_program_geodf = loc_program_df.set_geometry(loc_program_df['location'].apply(lambda p: shp.geometry.Point(p['coordinates'])), crs="EPSG:4269")
loc_program_geodf.head()

In [ ]:
program_endpoints_within = loc_program_geodf[loc_program_geodf.within(denver_boundary.geometry.iloc[0])]

In [ ]:
program_endpoints_within.head()

In [ ]:
fig, ax_arr = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(12,6))
denver_boundary.boundary.plot(ax=ax_arr[0])
program_endpoints_within.plot(column="program", cmap="tab10", ax=ax_arr[0])
denver_boundary.boundary.plot(ax=ax_arr[1])
program_endpoints_within.query('mode_confirm == "pilot_ebike"').plot(column="program", cmap="tab10", ax=ax_arr[1])
denver_boundary.boundary.plot(ax=ax_arr[2])
program_endpoints_within.query('mode_confirm == "drove_alone" | mode_confirm == "shared_ride" | mode_confirm == "taxi"').plot(column="program", cmap="tab10", ax=ax_arr[2])

In [ ]:
other_programs_endpoints_within_query = program_endpoints_within.query('program != "sc" & program != "prepilot" & program != "stage"')

In [ ]:
fig, ax_arr = plt.subplots(nrows=1, ncols=5, sharex=True, sharey=True, figsize=(20,10))
denver_boundary.boundary.plot(ax=ax_arr[0])
other_programs_endpoints_within_query.plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[0], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[0].set_title("all")
denver_boundary.boundary.plot(ax=ax_arr[1])
other_programs_endpoints_within_query.query('mode_confirm == "pilot_ebike"').plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[1], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[1].set_title("e-bike")
denver_boundary.boundary.plot(ax=ax_arr[2])
other_programs_endpoints_within_query.query('mode_confirm == "drove_alone"').plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[2], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[2].set_title("drove_alone")
denver_boundary.boundary.plot(ax=ax_arr[3])
other_programs_endpoints_within_query.query('mode_confirm == "shared_ride"').plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[3], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[3].set_title("shared_ride")
denver_boundary.boundary.plot(ax=ax_arr[4])
other_programs_endpoints_within_query.query('mode_confirm == "taxi"').plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[4], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[4].set_title("taxi")
# ax_arr[0].legend(["denver"] + other_programs_endpoints_within_query.program.unique().tolist(), loc="lower left", bbox_to_anchor=(0.5, 0.5))

In [ ]:
program_endpoints_within.to_file("ceo_results/program_endpoints_within_denver")

In [ ]:
program_endpoints_within.to_csv("ceo_results/program_endpoints_within_denver.csv")

In [ ]:
sc_pixel_stats_to_export = sc_pixel_stats.copy()

In [ ]:
sc_pixel_stats_to_export = sc_pixel_stats_to_export.astype({"e_bike_better": bool, "e_bike_gt_50_pct": bool})

In [ ]:
sc_pixel_stats_to_export.to_file("ceo_results/sc_pixel_stats")

In [ ]:
prepilot_pixel_stats_to_export = prepilot_pixel_stats.copy()

In [ ]:
prepilot_pixel_stats_to_export = prepilot_pixel_stats_to_export.astype({"e_bike_better": bool, "e_bike_gt_50_pct": bool})

In [ ]:
prepilot_pixel_stats_to_export.to_file("ceo_results/prepilot_pixel_stats")

In [ ]:
sc_pixel_stats.to_csv("ceo_results/sc_pixel_stats.csv")

In [ ]:
prepilot_pixel_stats.to_csv("ceo_results/prepilot_pixel_stats.csv")

In [ ]:
other_programs_pixel_stats_to_export = other_programs_pixel_stats.copy()

In [ ]:
other_programs_pixel_stats_to_export = other_programs_pixel_stats_to_export.astype({"e_bike_better": bool, "e_bike_gt_50_pct": bool})

In [ ]:
other_programs_pixel_stats_to_export.to_file("ceo_results/other_programs_pixel_stats")

In [ ]:
other_programs_pixel_stats.to_csv("ceo_results/other_programs_pixel_stats.csv")